# Fixed Income Derivatives Practice Problems and Notes

In [2]:
import pandas as pd
import numpy as np
from Binomial_Fixed import binomial, ratecurves
from Binomial_Fixed import treasury_cmds
from treasury_cmds import *
import scipy
from scipy.optimize import fsolve
from scipy.stats import norm
from datetime import date
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib as mpl
import datetime as dt
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'

It is July 30, 2015. The cheapest-to-deliver bond in a September 2015 Treasury bond futures contract is a 13% coupon bond, and delivery is expected to be made on September 30, 2015. Coupon payments on the bond are made on February 4 and August 4 each year. The term structure is flat, and the rate of interest with semiannual compounding is 12% per annum. The conversion factor for the bond is 1.5. The current quoted bond price is $110. Calculate the quoted futures price for the contract. Explain.

In [71]:
type('d')

str

In [74]:
def convert_dt(date):
    if type(date) != str:
        return date
    else:

        return pd.to_datetime(date)

In [26]:
(convert_dt("2015-01-01")-convert_dt("2014-08-15")).days

139

In [33]:
cpdate1 + pd.DateOffset(months=6)

Timestamp('2016-02-04 00:00:00')

In [116]:
p0 = 110
conversion = 1.5
repo = 2*np.log(1.06)
cpn = .13/2
date_0 = dt.date(2015,7,30)
cpdate = dt.date(2015,2,4)
cpdate1 = dt.date(2015,8,4)
delivery_dt = dt.date(2015,9,30)
period  = (cpdate1-cpdate).days
accint = ((date_0-cpdate).days/period)*(cpn)*100
cash_price =p0+accint
cash_price
cpdate1-date_0

def futures_price(clean_price, repo,cpn,curr_date, cpn_date0,cpn_date1,conversion,delivery_date):
    curr_date = convert_dt(curr_date)
    cpn_date0 = convert_dt(cpn_date0)
    cpn_date1 = convert_dt(cpn_date1)
    if cpn >.90:
        cpn = cpn*(1/100)
    else:
        cpn = cpn
    delivery_date = convert_dt(delivery_date)
    accint = ((curr_date-cpn_date0).days/(cpn_date1-cpn_date0).days)*(cpn*100)
    dirty_price = clean_price+accint
    I = cpn*100*(np.exp(-repo*(cpn_date1-curr_date).days/365))
    DTD = (delivery_date-curr_date).days/(365)
    discount = np.exp(repo*DTD)
    accrued_int = 100*cpn*((delivery_date-cpn_date1).days/((cpn_date1+pd.DateOffset(months = 6))-cpn_date1).days)
    F0 = (dirty_price-I)*discount - accrued_int

    return F0/conversion

def forward_price(clean_price, repo,cpn,curr_date,days_accrued,cpn_date1,delivery_date):
    curr_date = convert_dt(curr_date)
    cpn_date0 = curr_date - dt.timedelta(days = days_accrued)
    cpn_date1 = convert_dt(cpn_date1)
    delivery_date = convert_dt(delivery_date)

    repo = repo/100
    cpn = cpn*(1/2)

    accint = ((curr_date-cpn_date0).days/(cpn_date1-cpn_date0).days)*(cpn)
    dirty_price = clean_price+accint 

    I = cpn*(np.exp(-repo*(cpn_date1-curr_date).days/360))
    DTD = (delivery_date-curr_date).days/(360)
    discount = np.exp(repo*DTD)
    accrued_int = cpn*((delivery_date-cpn_date1).days/((cpn_date1+pd.DateOffset(months = 6))-cpn_date1).days)
    F0 = (dirty_price-cpn)*discount - accrued_int

    return F0





In [88]:
bond_info

,ticker,last_update_dt,px_last,maturity,days_to_mty,cpn,nxt_cpn_dt,days_to_next_coupon,int_acc,accrued_days_between_cpn_dates,days_acc,basis_mid,repo_implied_reporate,repo_reporate,conversion
0,91282CFZ Govt,2023-04-21,100.757812,2027-11-30,1681,3.875,2023-05-31,37,1.543613,182,145,13.138350,3.619994,4.815,0.9226
1,91282CGC Govt,2023-04-21,100.750000,2027-12-31,1712,3.875,2023-06-30,67,1.220304,181,114,17.307175,3.644369,4.815,0.9212
2,91282CGH Govt,2023-04-21,99.195312,2028-01-31,1743,3.500,2023-07-31,98,0.802486,181,83,24.180222,3.631045,4.815,0.9058
3,91282CGP Govt,2023-04-21,101.484375,2028-02-29,1772,4.000,2023-08-31,129,0.597826,184,55,32.893100,3.550495,4.815,0.9234
4,91282CGT Govt,2023-04-21,99.828125,2028-03-31,1803,3.625,2023-09-30,159,0.237705,183,24,38.039247,3.570506,4.815,0.9075


In [64]:
future_bonds = 'C:/Users/dcste/OneDrive/Fixed_Income_Derivatives/finm-fiderivs-2023/data/fut_bond_data_FVU3_2023-04-21.xlsx'
bdt_path = 'C:/Users/dcste/OneDrive/Fixed_Income_Derivatives/finm-fiderivs-2023/data/bdt_params_freq52_2023-04-21.xlsx'
future_description = pd.read_excel(future_bonds, sheet_name='future')
bond_info = pd.read_excel(future_bonds,sheet_name='bonds')
bdt_params = pd.read_excel(bdt_path)
Future_Price = future_description.iloc[1,1]


In [69]:
future_description

,field,FVU3 Comdty
0,last_update_dt,2023-04-21 00:00:00
1,px_last,109.789062
2,last_tradeable_dt,2023-09-29 00:00:00
3,fut_dlv_dt_last,2023-10-04 00:00:00
4,fut_days_expire,159
5,fut_ctd,T 3.875 11/30/27
6,fut_ctd_px,100.757812
7,fut_ctd_gross_basis,-17.074348
8,fut_ctd_net_basis,1.199828


In [67]:
DATE = '2023-04-21'
FUT_DAYS_EXPIRE = 159

In [70]:
Future_Expiration = convert_dt(DATE) +dt.timedelta(159)

In [100]:
bond_info

,ticker,last_update_dt,px_last,maturity,days_to_mty,cpn,nxt_cpn_dt,days_to_next_coupon,int_acc,accrued_days_between_cpn_dates,days_acc,basis_mid,repo_implied_reporate,repo_reporate,conversion
0,91282CFZ Govt,2023-04-21,100.757812,2027-11-30,1681,3.875,2023-05-31,37,1.543613,182,145,13.138350,3.619994,4.815,0.9226
1,91282CGC Govt,2023-04-21,100.750000,2027-12-31,1712,3.875,2023-06-30,67,1.220304,181,114,17.307175,3.644369,4.815,0.9212
2,91282CGH Govt,2023-04-21,99.195312,2028-01-31,1743,3.500,2023-07-31,98,0.802486,181,83,24.180222,3.631045,4.815,0.9058
3,91282CGP Govt,2023-04-21,101.484375,2028-02-29,1772,4.000,2023-08-31,129,0.597826,184,55,32.893100,3.550495,4.815,0.9234
4,91282CGT Govt,2023-04-21,99.828125,2028-03-31,1803,3.625,2023-09-30,159,0.237705,183,24,38.039247,3.570506,4.815,0.9075


In [117]:
fwd_price = bond_info.apply(lambda row:forward_price(clean_price=row['px_last'],repo = row['repo_reporate'],cpn = row['cpn'],curr_date=DATE,days_accrued=row['days_acc'],cpn_date1=row['nxt_cpn_dt'],delivery_date=Future_Expiration), axis = 1)

In [121]:
conversions = bond_info['conversion']

fwd_price - conversions*Future_Price


0   -0.055717
1    0.082618
2    0.347607
3    0.547646
4    0.757572
dtype: float64

In [113]:
Future_0 = futures_price(110,repo=repo,cpn = cpn,curr_date="2015-07-30",cpn_date0="2015-02-04",cpn_date1="2015-08-04",conversion=1.5,delivery_date="2015-09-30")

-0.01303720948843079